In [1]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

In [41]:
from shop.models import *
import json
from django.db.models import *
from django.db.models.functions import *


In [3]:
Order.objects.all()

<QuerySet []>

In [4]:
Order.objects.all()

<QuerySet [<Order: Order object (1)>, <Order: Order object (2)>, <Order: Order object (3)>, <Order: Order object (4)>, <Order: Order object (5)>, <Order: Order object (6)>, <Order: Order object (7)>, <Order: Order object (8)>, <Order: Order object (9)>, <Order: Order object (10)>, <Order: Order object (11)>, <Order: Order object (12)>, <Order: Order object (13)>, <Order: Order object (14)>, <Order: Order object (15)>, <Order: Order object (16)>, <Order: Order object (17)>, <Order: Order object (18)>, <Order: Order object (19)>, <Order: Order object (20)>, '...(remaining elements truncated)...']>

In [16]:
paym1 = Payment.objects.annotate(after_discount_price=F("price")-F("discount"))

In [17]:
pay_o = paym1.all().order_by("-after_discount_price")

In [18]:
for o in pay_o[:10]:
    print("ID:",o.id, ", PRICE:", o.price,", DISCOUNT:", o.discount, ", AFTER_DISCOUNT",o.after_discount_price)
    

ID: 92 , PRICE: 1200500.00 , DISCOUNT: 29433.25 , AFTER_DISCOUNT 1171066.75
ID: 82 , PRICE: 1200280.00 , DISCOUNT: 46229.40 , AFTER_DISCOUNT 1154050.60
ID: 137 , PRICE: 1200690.00 , DISCOUNT: 71407.25 , AFTER_DISCOUNT 1129282.75
ID: 105 , PRICE: 1200390.00 , DISCOUNT: 105019.11 , AFTER_DISCOUNT 1095370.89
ID: 45 , PRICE: 1218900.00 , DISCOUNT: 126859.95 , AFTER_DISCOUNT 1092040.05
ID: 7 , PRICE: 1201200.00 , DISCOUNT: 113446.20 , AFTER_DISCOUNT 1087753.80
ID: 18 , PRICE: 1202190.00 , DISCOUNT: 121922.64 , AFTER_DISCOUNT 1080267.36
ID: 77 , PRICE: 379000.00 , DISCOUNT: 19397.00 , AFTER_DISCOUNT 359603.00
ID: 127 , PRICE: 320450.00 , DISCOUNT: 14578.90 , AFTER_DISCOUNT 305871.10
ID: 125 , PRICE: 320399.00 , DISCOUNT: 17939.55 , AFTER_DISCOUNT 302459.45


In [20]:
pay_o2 = pay_o.filter(after_discount_price__gt=500000)
for o in pay_o2[:7]:
    print("ID:",o.id, ", PRICE:", o.price,", DISCOUNT:", o.discount, ", AFTER_DISCOUNT",o.after_discount_price)

ID: 92 , PRICE: 1200500.00 , DISCOUNT: 29433.25 , AFTER_DISCOUNT 1171066.75
ID: 82 , PRICE: 1200280.00 , DISCOUNT: 46229.40 , AFTER_DISCOUNT 1154050.60
ID: 137 , PRICE: 1200690.00 , DISCOUNT: 71407.25 , AFTER_DISCOUNT 1129282.75
ID: 105 , PRICE: 1200390.00 , DISCOUNT: 105019.11 , AFTER_DISCOUNT 1095370.89
ID: 45 , PRICE: 1218900.00 , DISCOUNT: 126859.95 , AFTER_DISCOUNT 1092040.05
ID: 7 , PRICE: 1201200.00 , DISCOUNT: 113446.20 , AFTER_DISCOUNT 1087753.80
ID: 18 , PRICE: 1202190.00 , DISCOUNT: 121922.64 , AFTER_DISCOUNT 1080267.36


In [35]:
cus1 = Customer.objects.annotate(
    full_name=Concat('first_name', Value(' '), 'last_name')
).order_by('full_name')[:5]
data = cus1.values('id', 'email', 'last_name', 'full_name')
print(data)

<QuerySet [{'id': 17, 'email': 'anantaya.deena@gmail.com', 'last_name': 'Tontong', 'full_name': 'Anantaya Tontong'}, {'id': 25, 'email': 'bancha.zeyou@gmail.com', 'last_name': 'Kittisompong', 'full_name': 'Bancha Kittisompong'}, {'id': 19, 'email': 'chayapol.231@gmail.com', 'last_name': 'Komprach', 'full_name': 'Chayapol Komprach'}, {'id': 4, 'email': 'dejwit.tt@gmail.com', 'last_name': 'Tangjareonsakul', 'full_name': 'Dejwit Tangjareonsakul'}, {'id': 11, 'email': 'jack.maa@gmail.com', 'last_name': 'Maa', 'full_name': 'Jack Maa'}]>


In [34]:
print(json.dumps(list(data), indent=4, sort_keys=False))

[
    {
        "id": 17,
        "first_name": "Anantaya",
        "last_name": "Tontong",
        "email": "anantaya.deena@gmail.com",
        "address": {
            "district": "Yan Nawa",
            "location": "60 Thanon Chan Road",
            "province": "Bangkok",
            "postal_code": 10120
        },
        "full_name": "Anantaya Tontong"
    },
    {
        "id": 25,
        "first_name": "Bancha",
        "last_name": "Kittisompong",
        "email": "bancha.zeyou@gmail.com",
        "address": {
            "district": "Watthana",
            "location": "6 Thong Lo Road",
            "province": "Bangkok",
            "postal_code": 10110
        },
        "full_name": "Bancha Kittisompong"
    },
    {
        "id": 19,
        "first_name": "Chayapol",
        "last_name": "Komprach",
        "email": "chayapol.231@gmail.com",
        "address": {
            "district": "Hang Chat",
            "location": "160 Lampang Road",
            "province": "Lampang

In [43]:
avg_pro = Product.objects.filter(remaining_amount__gt=0).aggregate(Avg("price", default=0))

In [45]:
avg_pro

{'price__avg': Decimal('29308.000000000000')}

In [49]:
sum_cart = CartItem.objects.filter(cart__create_date__month=5).aggregate(Sum("product__price"))

In [50]:
sum_cart

{'product__price__sum': Decimal('9912555.00')}

In [65]:
count_pro = Product.objects.filter(categories__name="Electronics", price__range=(8000, 50000)).aggregate(coconut=Count("remaining_amount"))
count_pro2 = Product.objects.filter(categories__name="Jewelry", price__range=(8000, 50000)).aggregate(coconut=Count("remaining_amount"))

In [83]:
print("PRODUCT CATEGORY NAME: Electronics", " PRODUCT COUNT:", count_pro['coconut'])
print("PRODUCT CATEGORY NAME: Jewelry", ", PRODUCT COUNT:", count_pro2['coconut'])



PRODUCT CATEGORY NAME: Electronics  PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry , PRODUCT COUNT: 1


{'coconut': 6}

In [74]:
last_order = Order.objects.filter(
    customer=OuterRef('pk')
).order_by('-order_date')

order = Customer.objects.annotate(newest_order=Subquery(last_order.values("order_date")[:1])).order_by("newest_order")
for i in order:
    print("CUSTOMER NAME:", i.first_name, i.last_name,"ORDER DATE:", i.newest_order)

CUSTOMER NAME: Sek Loso ORDER DATE: 2024-05-13
CUSTOMER NAME: Warit Pititat ORDER DATE: 2024-05-26
CUSTOMER NAME: Chayapol Komprach ORDER DATE: 2024-05-26
CUSTOMER NAME: Kwantip Aekopas ORDER DATE: 2024-05-28
CUSTOMER NAME: Kerikwan Laopong ORDER DATE: 2024-05-30
CUSTOMER NAME: Pattanapoom Srisawangjan ORDER DATE: 2024-05-30
CUSTOMER NAME: Manit Senapan ORDER DATE: 2024-06-06
CUSTOMER NAME: Techathorn Sriwanwilai ORDER DATE: 2024-06-07
CUSTOMER NAME: Anantaya Tontong ORDER DATE: 2024-06-08
CUSTOMER NAME: Kittitorn Wilairat ORDER DATE: 2024-06-11
CUSTOMER NAME: Porntipa Pasakul ORDER DATE: 2024-06-12
CUSTOMER NAME: Peeraya Wichienchan ORDER DATE: 2024-06-13
CUSTOMER NAME: Rachanon Hongpatra ORDER DATE: 2024-06-14
CUSTOMER NAME: Suwichada Jarastham ORDER DATE: 2024-06-14
CUSTOMER NAME: Pong Sawadiwong ORDER DATE: 2024-06-15
CUSTOMER NAME: Dejwit Tangjareonsakul ORDER DATE: 2024-06-16
CUSTOMER NAME: Bancha Kittisompong ORDER DATE: 2024-06-17
CUSTOMER NAME: Yanisa Nantapak ORDER DATE: 2024

In [125]:
customer1 = Customer.objects.get(first_name="Sek")
order1 = Order(
    customer=customer1
    ,
    order_date='2025-02-01'
)
order1.save()
order2 = Order(
    customer=customer1,
    order_date='2025-02-02'
)
order1.save()
order2.save()

pro1 = Product.objects.get(name="Bird Cage")
pro2 = Product.objects.get(name="Microwave Oven")

ori1 = OrderItem(
    order=order1,
    product=pro1,
)
ori2= OrderItem(
    order=order2,
    product=pro2,
)
ori1.save()
ori2.save()

paym1 = Payment(
    order=order1,
    payment_date='2025-01-01',
    price=1699,
    discount=112.27
)
paym2 = Payment(
    order=order2,
    payment_date='2025-01-01',
    price=1740,
    discount=140.81
)
paym1.save()
paym2.save()
pay1 = PaymentItem(
    payment=paym1,
    order_item=ori1,
    price=1699,
    discount=112.27
)
pay2 = PaymentItem(
    payment=paym2,
    order_item=ori2,
    price=1740,
    discount=140.81
)
o1 = Order.objects.get(pk=184)
o2 = Order.objects.get(pk=185)
o3 = Order.objects.get(pk=186)

pay3 = Payment(
    order=o1,
    payment_date='2025-01-01',
    price=1740,
    discount=140.81
)
pay4 = Payment(
    order=o2,
    payment_date='2025-01-01',
    price=1699,
    discount=112.27
)
pay5 = Payment(
    order=o3,
    payment_date='2025-01-01',
    price=1740,
    discount=140.81
)

pay3.save()
pay4.save()
pay5.save()

pay1.save()
pay2.save()

In [130]:
o1 = Order.objects.get(pk=184)
o2 = Order.objects.get(pk=185)
o3 = Order.objects.get(pk=186)



sek = OrderItem.objects.filter(order__customer__first_name="Sek")
# print(sek)
for i in sek:
    print("CUSTOMER NAME: ",i.order.customer.first_name, "ORDER ID:", i.order.id, "PRICE:",i.order.payment.price, "DISCOUNT",i.order.payment.discount,"PRODUCT:",i.product.name)

CUSTOMER NAME:  Sek ORDER ID: 48 PRICE: 1699.00 DISCOUNT 112.27 PRODUCT: Bird Cage
CUSTOMER NAME:  Sek ORDER ID: 48 PRICE: 1699.00 DISCOUNT 112.27 PRODUCT: Hot Wheels Track Builder System Race Crate
CUSTOMER NAME:  Sek ORDER ID: 59 PRICE: 1740.00 DISCOUNT 140.81 PRODUCT: Microwave Oven
CUSTOMER NAME:  Sek ORDER ID: 59 PRICE: 1740.00 DISCOUNT 140.81 PRODUCT: Men's Polo Shirt
CUSTOMER NAME:  Sek ORDER ID: 186 PRICE: 1740.00 DISCOUNT 140.81 PRODUCT: Bird Cage
CUSTOMER NAME:  Sek ORDER ID: 187 PRICE: 1740.00 DISCOUNT 140.81 PRODUCT: Microwave Oven
CUSTOMER NAME:  Sek ORDER ID: 188 PRICE: 1699.00 DISCOUNT 112.27 PRODUCT: Bird Cage
CUSTOMER NAME:  Sek ORDER ID: 189 PRICE: 1740.00 DISCOUNT 140.81 PRODUCT: Microwave Oven
CUSTOMER NAME:  Sek ORDER ID: 190 PRICE: 1699.00 DISCOUNT 112.27 PRODUCT: Bird Cage
CUSTOMER NAME:  Sek ORDER ID: 191 PRICE: 1740.00 DISCOUNT 140.81 PRODUCT: Microwave Oven
CUSTOMER NAME:  Sek ORDER ID: 192 PRICE: 1699.00 DISCOUNT 112.27 PRODUCT: Bird Cage
CUSTOMER NAME:  Sek 

In [137]:
from django.db.models import F, ExpressionWrapper, DecimalField, Count
import json


payments = Payment.objects.filter(
    paymentmethod__method='CREDIT'
).annotate(
    after_discount=(
        F('price') - F('discount')
    ),
    item_count=Count('paymentitem')
).filter(after_discount__lt=500)

for p in payments:
    print("PRODUCT ID", p.id, "AFTER_DISCOUNT", p.after_discount, "ITEM COUNT", p.item_count)


PRODUCT ID 40 AFTER_DISCOUNT 493.06 ITEM COUNT 2
PRODUCT ID 13 AFTER_DISCOUNT 448.54 ITEM COUNT 2


In [154]:
p=Product.objects.filter(Q(categories__name="Information Technology"), Q(categories__name="Electronics and Toys")).distinct().order_by("id")[:10]

for i in p:
    ca = i.categories.all()
    print(i.id, i.name ,[n.name for n in ca])

In [142]:
cat = ProductCategory.objects.get(name="Electronics")
cat.name = "Information Technology"
cat.save()

In [144]:
toy1 = Product.objects.filter(categories__name="Information Technology")
toy2 = ProductCategory.objects.filter(name="Toys and Games").first()

for i in toy1:
    i.categories.remove(toy2)
    i.categories.add(cat)
    
toy2.delete()

(11, {'shop.Product_categories': 10, 'shop.ProductCategory': 1})

In [145]:
Product.objects.filter(categories__name="Electronics and Toys")

<QuerySet []>

In [146]:
Product.objects.filter(categories__name="Electronics and Toys").count()

0